In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import datetime


from sklearn.linear_model import LinearRegression


In [2]:
a = pd.read_csv("availability.csv")
b = pd.read_csv("currentweather.csv")

In [3]:
b.head(5)

dt description  wind_speed  humidity  temperature  pressure
0  1551726218        Rain           4        80            4       994
1  1551731330      Clouds           4        80            4       994
2  1551734153        Rain           5       100            4       994
3  1551735821        Rain           6       100            4       993
4  1551738525        Rain           6        80            4       993

In [4]:
# remove redundant columns from currentweather. We will not be using these columns in our predictive model
b = b.drop(["wind_speed", "humidity", "pressure", "temperature"], axis=1)

In [5]:
#change datestamp column name in second df so it matches datestamp in first df
b.rename(columns={"dt": "last_update"}, inplace=True)

In [6]:
#convert description of weather into categorical feature
b['description'] = b['description'].astype('category')
b.dtypes

last_update       int64
description    category
dtype: object

In [7]:
a.head(5)

number          last_update  available_bike_stands  available_bikes status  \
0       2  2019-03-04 19:09:57                     15                5   OPEN   
1       2  2019-03-04 19:11:16                     16                4   OPEN   
2       2  2019-03-04 19:17:33                     17                3   OPEN   
3       2  2019-03-04 19:21:48                     17                3   OPEN   
4       2  2019-03-04 19:29:58                     16                4   OPEN   

   banking  
0        1  
1        1  
2        1  
3        1  
4        1

In [8]:
# remove redundant columns from currentweather. We will not be using these columns in our predictive model
a = a.drop(["status", "banking"], axis=1)

In [9]:
# CHECK FOR DUPLICATES. If data scraping correctly, there shouldn't be any.
# No duplicate rows.
print('Duplicate rows in availability:', a.duplicated()[a.duplicated() == True].shape[0])
# No duplicate columns.
print('Duplicate columns in availability:',a.columns.size - a.columns.unique().size)
print("Total rows:", a.shape[0])
print()
# No duplicate rows.
print('Duplicate rows in currentweather:', b.duplicated()[b.duplicated() == True].shape[0])
# No duplicate columns.
print('Duplicate columns in currentweather:',b.columns.size - b.columns.unique().size)
print("Total rows:", b.shape[0])

Duplicate rows in availability: 0
Duplicate columns in availability: 0
Total rows: 821475

Duplicate rows in currentweather: 0
Duplicate columns in currentweather: 0
Total rows: 1034


In [10]:
b['description'].value_counts()

Clouds     589
Rain       359
Drizzle     39
Clear       29
Mist        17
Fog          1
Name: description, dtype: int64

In [11]:
# drop row with 'fog' as weather description as there is only 1
b = b[b.description != 'Fog']

In [12]:
b['description'].value_counts()

Clouds     589
Rain       359
Drizzle     39
Clear       29
Mist        17
Fog          0
Name: description, dtype: int64

In [13]:
# change datatype to datetime so that we can change the format from epoch to yyyy-mm-dd hh-mm-ss
b['last_update'] = pd.to_datetime(b['last_update'],unit='s')

In [14]:
# change formatting, round to nearest hour
b['last_update'] = b['last_update'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour, 60*(dt.minute // 60)))

In [15]:
# check how many rows before change
b.shape[0]

1033

In [16]:
# remove rows where timestamp is the same after rounding so that there is just one row per hour in currentweather
b = b.sort_values('last_update', ascending=True)
b = b.drop_duplicates(subset='last_update', keep='first')

In [17]:
# check how many rows after change
b.shape[0]

804

In [18]:
# change datatype to datetime so that we can change the format from epoch to yyyy-mm-dd hh-mm-ss
#a['last_update'] = pd.to_datetime(a['last_update'],unit='s')

# change datatype to datetime so that we can change the format from epoch to yyyy-mm-dd hh-mm-ss
a['last_update'] = pd.to_datetime(a['last_update'])

In [19]:
# change formatting, round to nearest hour as we did above
a['last_update'] = a['last_update'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour, 60*(dt.minute // 60)))

In [20]:
# check how many rows before change
a.shape[0]

821475

In [21]:
# remove rows where timestamp and station number are the same. there will be many as data was scraping every 5 mins
a = a.drop_duplicates(['last_update', 'number'])

In [22]:
a.shape[0]

105751

In [23]:
a['total_bike_stands'] = a['available_bikes'] + a['available_bike_stands']

In [24]:
# merge the tables into a new df
merged = a.merge(b, on='last_update')

In [25]:
a.head(5)

number         last_update  available_bike_stands  available_bikes  \
0        2 2019-03-04 19:00:00                     15                5   
9        2 2019-03-04 20:00:00                     14                6   
18       2 2019-03-04 21:00:00                     14                6   
23       2 2019-03-04 22:00:00                     15                5   
32       2 2019-03-04 23:00:00                      7               13   

    total_bike_stands  
0                  20  
9                  20  
18                 20  
23                 20  
32                 20

In [26]:
# sort the values in merged based first on their timestamp, then on station number to ensure that this has worked
merged = merged.sort_values(['last_update', 'number'], ascending=True)

In [27]:
# looks as if there are no duplicates station numbers in each hour slot
merged

number         last_update  available_bike_stands  available_bikes  \
0           2 2019-03-04 19:00:00                     15                5   
1           3 2019-03-04 19:00:00                     19                1   
2           4 2019-03-04 19:00:00                     20                0   
3           5 2019-03-04 19:00:00                     11               29   
4           6 2019-03-04 19:00:00                     16                4   
5           7 2019-03-04 19:00:00                      9               19   
6           8 2019-03-04 19:00:00                     15               15   
7           9 2019-03-04 19:00:00                      5               19   
8          10 2019-03-04 19:00:00                      7                9   
9          11 2019-03-04 19:00:00                     30                0   
10         12 2019-03-04 19:00:00                     16                4   
11         13 2019-03-04 19:00:00                     29                1   
12         14 2019-03-04 19:00:00                      5               25   
13         15 2019-03-04 19:00:00                     16                0   
14         16 2019-03-04 19:00:00                      4               16   
15         17 2019-03-04 19:00:00                     18                2   
16         18 2019-03-04 19:00:00                     24                6   
17         19 2019-03-04 19:00:00                     30                0   
18         21 2019-03-04 19:00:00                      4               26   
19         22 2019-03-04 19:00:00                      1               19   
20         23 2019-03-04 19:00:00                      2               28   
21         24 2019-03-04 19:00:00                     14                6   
22         25 2019-03-04 19:00:00                     28                2   
23         26 2019-03-04 19:00:00                      3               17   
24         27 2019-03-04 19:00:00                     18                2   
25         28 2019-03-04 19:00:00                     26                4   
26         29 2019-03-04 19:00:00                     23                6   
27         30 2019-03-04 19:00:00                     18                2   
28         31 2019-03-04 19:00:00                     14                6   
29         32 2019-03-04 19:00:00                      7               23   
...       ...                 ...                    ...              ...   
90532      86 2019-04-13 07:00:00                      3               35   
90533      87 2019-04-13 07:00:00                     12               26   
90534      88 2019-04-13 07:00:00                      4               26   
90535      89 2019-04-13 07:00:00                     38                1   
90536      90 2019-04-13 07:00:00                     35                5   
90537      91 2019-04-13 07:00:00                     10               20   
90538      92 2019-04-13 07:00:00                      0               40   
90539      93 2019-04-13 07:00:00                      0               40   
90540      94 2019-04-13 07:00:00                     11               29   
90541      95 2019-04-13 07:00:00                      7               33   
90542      96 2019-04-13 07:00:00                     15               15   
90543      97 2019-04-13 07:00:00                     13               27   
90544      98 2019-04-13 07:00:00                     40                0   
90545      99 2019-04-13 07:00:00                     28                2   
90546     100 2019-04-13 07:00:00                      0               25   
90547     101 2019-04-13 07:00:00                     22                8   
90548     102 2019-04-13 07:00:00                     12               28   
90549     103 2019-04-13 07:00:00                     39                1   
90550     104 2019-04-13 07:00:00                     26               14   
90551     105 2019-04-13 07:00:00                     25               11   
905

In [28]:
merged['last_update'] = (merged['last_update'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [29]:
merged.head(5)

number  last_update  available_bike_stands  available_bikes  \
0       2   1551726000                     15                5   
1       3   1551726000                     19                1   
2       4   1551726000                     20                0   
3       5   1551726000                     11               29   
4       6   1551726000                     16                4   

   total_bike_stands description  
0                 20        Rain  
1                 20        Rain  
2                 20        Rain  
3                 40        Rain  
4                 20        Rain

In [30]:
# save to CSV
merged.to_csv('weatheravailability.csv')